In [1]:
import pandas as pd
import numpy as np
import re
#import matplotlib.pyplot as plt
import datetime

In [2]:
ratings_ba = pd.read_csv('ratings_ba.csv', keep_default_na=False)

In [3]:
ratings_rb = pd.read_csv('ratings_rb.csv', keep_default_na=False)

In [4]:
beer_ba = pd.read_csv('./BeerAdvocate/beers.csv')

In [5]:
beer_rb = pd.read_csv('./RateBeer/beers.csv')

In [6]:
users = pd.read_csv('users.csv')

In [7]:
breweries = pd.read_csv('brew.csv')

In [8]:
# Data here: https://www.kaggle.com/datasets/rtatman/english-word-frequency
#word_freq = pd.read_csv('unigram_freq.csv')

In [9]:
ratings_ba

,beer_name,beer_id,date,user_name,user_id,overall,text
0,Régab,142544,1440064800,nmann08,nmann08.184925,3.0,"From a bottle, pours a piss yellow color with ..."
1,Barelegs Brew,19590,1235127600,StJamesGate,stjamesgate.163714,3.5,Pours pale copper with a thin head that quickl...
2,Barelegs Brew,19590,1142247600,mdagnew,mdagnew.19527,3.5,"500ml Bottle bought from The Vintage, Antrim....."
3,Barelegs Brew,19590,1101898800,helloloser12345,helloloser12345.10867,4.5,Serving: 500ml brown bottlePour: Good head wit...
4,Barelegs Brew,19590,1093860000,cypressbob,cypressbob.3708,4.0,"500ml bottlePours with a light, slightly hazy ..."
...,...,...,...,...,...,...,...
2672671,Irvington Pale Ale,19141,1100516400,ecoboy,ecoboy.1641,3.5,"It may say APA on this site, but talking to th..."
2672672,Kölsch Ale,19139,1141038000,jaluria,jaluria.31362,4.0,I'll start off reviewing this beer with the fo...
2672673,Kölsch Ale,19139,1140346800,Dithyramb,dithyramb.4413,3.0,Served on draft at the pub. Poured a clear yel...
2672674,Kölsch Ale,19139,1092132000,ManekiNeko,manekineko.2654,1.5,Served on tap at the brewpub in a pint glass.A...


In [10]:
users

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
0,7820,465.0,nmann08.184925,nmann08,1.199704e+09,"United States, Washington"
1,2521,2504.0,stjamesgate.163714,StJamesGate,1.191838e+09,"United States, New York"
2,1797,1143.0,mdagnew.19527,mdagnew,1.116410e+09,Northern Ireland
3,31,31.0,helloloser12345.10867,helloloser12345,1.101380e+09,Northern Ireland
4,604,604.0,cypressbob.3708,cypressbob,1.069326e+09,Northern Ireland
...,...,...,...,...,...,...
223873,1,NaN,134893,stamfordbus,1.315217e+09,England
223874,1,NaN,327816,fobia405,1.406282e+09,Belgium
223875,1,NaN,183139,junkin,1.339841e+09,NaN
223876,3,NaN,82020,klesidra,1.222250e+09,Slovenia


In [11]:
breweries

,id,location,name,nbr_beers
0,39912,Kyrgyzstan,Abdysh-Ata (Абдыш Ата),5
1,39913,Kyrgyzstan,Arpa (АРПА),1
2,39914,Kyrgyzstan,Bear Beer,4
3,39915,Kyrgyzstan,Blonder Pub,4
4,39916,Kyrgyzstan,Kellers Bier,2
...,...,...,...,...
33511,128,Germany,Privatbrauerei Franz Joseph Sailer,6
33512,5023,Aruba,Aruba Brewing,0
33513,1573,"United States, New York",John Harvard’s Brewery & Ale House,491
33514,16186,"United States, North Carolina",Triangle Brewing Company,20


### Length of Review

In [12]:
# Regex to replace all non alphabet/space characters with a space
# This is not a perfect 100% encompassing solution. Users sometimes have typos or other irregularities in their text e.g. 'bottlePour' being counted as 1 word.
# Phrases such as 0.33ml might also be separated into two mistakenly such as '0' and '33ml'
def text_only(review): 
    return re.sub('[^a-zA-Z ]+', ' ', review)

In [13]:
ba_raw_text = ratings_ba['text'].apply(text_only)

In [14]:
ba_raw_text

0          From a bottle  pours a piss yellow color with ...
1          Pours pale copper with a thin head that quickl...
2           ml Bottle bought from The Vintage  Antrim Pou...
3          Serving   ml brown bottlePour  Good head with ...
4           ml bottlePours with a light  slightly hazy go...
                                 ...                        
2672671    It may say APA on this site  but talking to th...
2672672    I ll start off reviewing this beer with the fo...
2672673    Served on draft at the pub  Poured a clear yel...
2672674    Served on tap at the brewpub in a pint glass A...
2672675    Dark brown brew served on tap at the brewpub  ...
Name: text, Length: 2672676, dtype: object

In [15]:
def count_words(review):
    return len(review.split())

In [16]:
review_len_ba = ba_raw_text.apply(count_words)

In [17]:
review_len_ba

0           61
1           47
2           72
3           82
4           42
          ... 
2672671    153
2672672    165
2672673     48
2672674    111
2672675     65
Name: text, Length: 2672676, dtype: int64

In [18]:
ratings_ba['text_length'] = review_len_ba

In [19]:
ratings_ba

,beer_name,beer_id,date,user_name,user_id,overall,text,text_length
0,Régab,142544,1440064800,nmann08,nmann08.184925,3.0,"From a bottle, pours a piss yellow color with ...",61
1,Barelegs Brew,19590,1235127600,StJamesGate,stjamesgate.163714,3.5,Pours pale copper with a thin head that quickl...,47
2,Barelegs Brew,19590,1142247600,mdagnew,mdagnew.19527,3.5,"500ml Bottle bought from The Vintage, Antrim.....",72
3,Barelegs Brew,19590,1101898800,helloloser12345,helloloser12345.10867,4.5,Serving: 500ml brown bottlePour: Good head wit...,82
4,Barelegs Brew,19590,1093860000,cypressbob,cypressbob.3708,4.0,"500ml bottlePours with a light, slightly hazy ...",42
...,...,...,...,...,...,...,...,...
2672671,Irvington Pale Ale,19141,1100516400,ecoboy,ecoboy.1641,3.5,"It may say APA on this site, but talking to th...",153
2672672,Kölsch Ale,19139,1141038000,jaluria,jaluria.31362,4.0,I'll start off reviewing this beer with the fo...,165
2672673,Kölsch Ale,19139,1140346800,Dithyramb,dithyramb.4413,3.0,Served on draft at the pub. Poured a clear yel...,48
2672674,Kölsch Ale,19139,1092132000,ManekiNeko,manekineko.2654,1.5,Served on tap at the brewpub in a pint glass.A...,111


### Relative time

In [20]:
# Finds the first review time for each beer
first_review_time_ba = ratings_ba.groupby('beer_name')['date'].min()

In [21]:
first_review_time_ba

beer_name
!                             1460196000
! (Old Ale)                   1271671200
!!!PA Simcoe Mosaic           1480417200
!!C&G                         1497002400
!Pablamo!                     1394535600
                                 ...    
台.P.A. (台 Pale Ale)           1433671200
日本から来たネコ                      1500458400
横須賀ビアサケ (Yokosuka Biasake)    1308996000
葉山ビール (Hayama Beer)           1308823200
﻿Boji Blue Pale Ale           1332586800
Name: date, Length: 148819, dtype: int64

In [22]:
# Relative time differential in unixtime
def relative_time_ba(time, beer_name):
    return time - first_review_time_ba.loc[beer_name]


In [23]:
ratings_ba['relative_time'] = ratings_ba.apply(lambda x: relative_time_ba(x['date'], x['beer_name']), axis=1)

In [24]:
ratings_ba

,beer_name,beer_id,date,user_name,user_id,overall,text,text_length,relative_time
0,Régab,142544,1440064800,nmann08,nmann08.184925,3.0,"From a bottle, pours a piss yellow color with ...",61,0
1,Barelegs Brew,19590,1235127600,StJamesGate,stjamesgate.163714,3.5,Pours pale copper with a thin head that quickl...,47,141267600
2,Barelegs Brew,19590,1142247600,mdagnew,mdagnew.19527,3.5,"500ml Bottle bought from The Vintage, Antrim.....",72,48387600
3,Barelegs Brew,19590,1101898800,helloloser12345,helloloser12345.10867,4.5,Serving: 500ml brown bottlePour: Good head wit...,82,8038800
4,Barelegs Brew,19590,1093860000,cypressbob,cypressbob.3708,4.0,"500ml bottlePours with a light, slightly hazy ...",42,0
...,...,...,...,...,...,...,...,...,...
2672671,Irvington Pale Ale,19141,1100516400,ecoboy,ecoboy.1641,3.5,"It may say APA on this site, but talking to th...",153,0
2672672,Kölsch Ale,19139,1141038000,jaluria,jaluria.31362,4.0,I'll start off reviewing this beer with the fo...,165,78109200
2672673,Kölsch Ale,19139,1140346800,Dithyramb,dithyramb.4413,3.0,Served on draft at the pub. Poured a clear yel...,48,77418000
2672674,Kölsch Ale,19139,1092132000,ManekiNeko,manekineko.2654,1.5,Served on tap at the brewpub in a pint glass.A...,111,29203200


### Time of writing review

In [25]:
# Same column as 'date' but in a more human-readable form
ratings_ba['review_time'] = ratings_ba['date'].apply(datetime.datetime.fromtimestamp)

In [26]:
ratings_ba

,beer_name,beer_id,date,user_name,user_id,overall,text,text_length,relative_time,review_time
0,Régab,142544,1440064800,nmann08,nmann08.184925,3.0,"From a bottle, pours a piss yellow color with ...",61,0,2015-08-20 12:00:00
1,Barelegs Brew,19590,1235127600,StJamesGate,stjamesgate.163714,3.5,Pours pale copper with a thin head that quickl...,47,141267600,2009-02-20 12:00:00
2,Barelegs Brew,19590,1142247600,mdagnew,mdagnew.19527,3.5,"500ml Bottle bought from The Vintage, Antrim.....",72,48387600,2006-03-13 12:00:00
3,Barelegs Brew,19590,1101898800,helloloser12345,helloloser12345.10867,4.5,Serving: 500ml brown bottlePour: Good head wit...,82,8038800,2004-12-01 12:00:00
4,Barelegs Brew,19590,1093860000,cypressbob,cypressbob.3708,4.0,"500ml bottlePours with a light, slightly hazy ...",42,0,2004-08-30 12:00:00
...,...,...,...,...,...,...,...,...,...,...
2672671,Irvington Pale Ale,19141,1100516400,ecoboy,ecoboy.1641,3.5,"It may say APA on this site, but talking to th...",153,0,2004-11-15 12:00:00
2672672,Kölsch Ale,19139,1141038000,jaluria,jaluria.31362,4.0,I'll start off reviewing this beer with the fo...,165,78109200,2006-02-27 12:00:00
2672673,Kölsch Ale,19139,1140346800,Dithyramb,dithyramb.4413,3.0,Served on draft at the pub. Poured a clear yel...,48,77418000,2006-02-19 12:00:00
2672674,Kölsch Ale,19139,1092132000,ManekiNeko,manekineko.2654,1.5,Served on tap at the brewpub in a pint glass.A...,111,29203200,2004-08-10 12:00:00


### Country of beer and country of reviewer

In [27]:
user_df = users[['user_id', 'location']].set_index(keys='user_id')

In [28]:
user_df

,location
user_id,
nmann08.184925,"United States, Washington"
stjamesgate.163714,"United States, New York"
mdagnew.19527,Northern Ireland
helloloser12345.10867,Northern Ireland
cypressbob.3708,Northern Ireland
...,...
134893,England
327816,Belgium
183139,NaN


In [29]:
# Assign each review with their corresponding user's location
ratings_ba = ratings_ba.merge(how='left',right=user_df, on='user_id')

In [31]:
ratings_ba

,beer_name,beer_id,date,user_name,user_id,overall,text,text_length,relative_time,review_time,location
0,Régab,142544,1440064800,nmann08,nmann08.184925,3.0,"From a bottle, pours a piss yellow color with ...",61,0,2015-08-20 12:00:00,"United States, Washington"
1,Barelegs Brew,19590,1235127600,StJamesGate,stjamesgate.163714,3.5,Pours pale copper with a thin head that quickl...,47,141267600,2009-02-20 12:00:00,"United States, New York"
2,Barelegs Brew,19590,1142247600,mdagnew,mdagnew.19527,3.5,"500ml Bottle bought from The Vintage, Antrim.....",72,48387600,2006-03-13 12:00:00,Northern Ireland
3,Barelegs Brew,19590,1101898800,helloloser12345,helloloser12345.10867,4.5,Serving: 500ml brown bottlePour: Good head wit...,82,8038800,2004-12-01 12:00:00,Northern Ireland
4,Barelegs Brew,19590,1093860000,cypressbob,cypressbob.3708,4.0,"500ml bottlePours with a light, slightly hazy ...",42,0,2004-08-30 12:00:00,Northern Ireland
...,...,...,...,...,...,...,...,...,...,...,...
2672671,Irvington Pale Ale,19141,1100516400,ecoboy,ecoboy.1641,3.5,"It may say APA on this site, but talking to th...",153,0,2004-11-15 12:00:00,"United States, Rhode Island"
2672672,Kölsch Ale,19139,1141038000,jaluria,jaluria.31362,4.0,I'll start off reviewing this beer with the fo...,165,78109200,2006-02-27 12:00:00,"United States, Connecticut"
2672673,Kölsch Ale,19139,1140346800,Dithyramb,dithyramb.4413,3.0,Served on draft at the pub. Poured a clear yel...,48,77418000,2006-02-19 12:00:00,"United States, Maryland"
2672674,Kölsch Ale,19139,1092132000,ManekiNeko,manekineko.2654,1.5,Served on tap at the brewpub in a pint glass.A...,111,29203200,2004-08-10 12:00:00,"United States, Virginia"


In [32]:
beer_to_brew_ba = beer_ba[['beer_id', 'brewery_id']].set_index(keys='beer_id')['brewery_id']

In [33]:
brew_locations = breweries[['id', 'location']].set_index(keys='id')['location']


In [34]:
# Assign each review with the corrresponding location to where it was brewed
def beer_country_ba(identifier):
    brewery = beer_to_brew_ba.loc[identifier]
    return brew_locations.loc[brewery].values[0]


In [35]:
ratings_ba['beer_location'] = ratings_ba['beer_id'].apply(beer_country_ba)

In [36]:
ratings_ba

,beer_name,beer_id,date,user_name,user_id,overall,text,text_length,relative_time,review_time,location,beer_location
0,Régab,142544,1440064800,nmann08,nmann08.184925,3.0,"From a bottle, pours a piss yellow color with ...",61,0,2015-08-20 12:00:00,"United States, Washington",Gabon
1,Barelegs Brew,19590,1235127600,StJamesGate,stjamesgate.163714,3.5,Pours pale copper with a thin head that quickl...,47,141267600,2009-02-20 12:00:00,"United States, New York",Northern Ireland
2,Barelegs Brew,19590,1142247600,mdagnew,mdagnew.19527,3.5,"500ml Bottle bought from The Vintage, Antrim.....",72,48387600,2006-03-13 12:00:00,Northern Ireland,Northern Ireland
3,Barelegs Brew,19590,1101898800,helloloser12345,helloloser12345.10867,4.5,Serving: 500ml brown bottlePour: Good head wit...,82,8038800,2004-12-01 12:00:00,Northern Ireland,Northern Ireland
4,Barelegs Brew,19590,1093860000,cypressbob,cypressbob.3708,4.0,"500ml bottlePours with a light, slightly hazy ...",42,0,2004-08-30 12:00:00,Northern Ireland,Northern Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...
2672671,Irvington Pale Ale,19141,1100516400,ecoboy,ecoboy.1641,3.5,"It may say APA on this site, but talking to th...",153,0,2004-11-15 12:00:00,"United States, Rhode Island","United States, Maryland"
2672672,Kölsch Ale,19139,1141038000,jaluria,jaluria.31362,4.0,I'll start off reviewing this beer with the fo...,165,78109200,2006-02-27 12:00:00,"United States, Connecticut","United States, Maryland"
2672673,Kölsch Ale,19139,1140346800,Dithyramb,dithyramb.4413,3.0,Served on draft at the pub. Poured a clear yel...,48,77418000,2006-02-19 12:00:00,"United States, Maryland","United States, Maryland"
2672674,Kölsch Ale,19139,1092132000,ManekiNeko,manekineko.2654,1.5,Served on tap at the brewpub in a pint glass.A...,111,29203200,2004-08-10 12:00:00,"United States, Virginia","United States, Maryland"


### Beer popularity

In [37]:
rb_counts = ratings_rb['beer_name'].value_counts()

In [38]:
ba_counts = ratings_ba['beer_name'].value_counts()

In [39]:
beer_popularity = pd.DataFrame(data=rb_counts.add(ba_counts, fill_value=0).sort_values(ascending=False)).rename(columns={'beer_name' : 'review_count'})

In [40]:
beer_popularity

,review_count
Guinness Draught,7593.0
Arrogant Bastard Ale,7405.0
St. Bernardus Abt 12,7348.0
Brooklyn Black Chocolate Stout,6251.0
Ayinger Celebrator Doppelbock,6200.0
...,...
SKA Insider Apple Ale,1.0
SKA Local Series #01 (Spiced Belgian-Style Blonde),1.0
SKA Local Series #02/09 (Double Chocolate Stout),1.0
SKA Local Series #04 (Watermelon Wheat),1.0
